# Heart Rate Forecasting

In [ ]:
# Clone current repository and install dependencies
!git clone https://github.com/allandbb/heartrate-forecasting.git
%cd heartrate-forecasting
!pip install -q -r requirements.txt

In [ ]:
# Check if GPU is available
import torch

print("GPU available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))

GPU disponible: False


In [ ]:
import utils
from sklearn.model_selection import train_test_split

folder_path = 'dataset/'
dfSelectedMean = utils.loadAllFiles(folder_path)

# This is the input and output size for the forecasting model. You can adjust these values as needed.
inp = 200
out = 200

# Split columns of dataset
df_70, df_30 = utils.selectRandomColumns(dfSelectedMean)

# Estandarización
pathEst_70 = 'dataset/values_deses_70.csv'
pathEst_30 = 'dataset/values_deses_30.csv'

df_scaled_70, values_deses_70 = utils.estandarizar(df_70, pathEst_70)
df_scaled_30, values_deses_30 = utils.estandarizar(df_30, pathEst_30)

# Split in input and output
X_train, y_train, ids_list_tr = utils.series_to_supervised_matrix(
    df_scaled_70.iloc[:, :-2], input_size=inp, output_size=out
)

X_tr, X_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

X_test, y_test, ids_list_te = utils.series_to_supervised_matrix(
    df_scaled_30.iloc[:, :-2], input_size=inp, output_size=out
)

print("Datasets splitted and ready!")
print("Shapes:", X_tr.shape, y_tr.shape, X_va.shape, y_va.shape, X_test.shape, y_test.shape)


In [ ]:
# Chronos forecasting
import os
import sys

os.chdir("/content/heartrate-forecasting")
sys.path.insert(0, "/content/heartrate-forecasting")

from wrappers.ChronosSupervisedWrapper import ChronosSupervisedWrapper

wrapper = ChronosSupervisedWrapper(config_path="configs/chronos_config.yaml")
y_pred_smoke = wrapper.predict(X_test[:5])

print("Predictions shape:", y_pred_smoke.shape)
print("Predictions (first 5 samples):", y_pred_smoke)

# Evaluación completa sobre test set
metrics = wrapper.evaluate(X_test, y_test)

# Evaluación sobre validation set
metrics_val = wrapper.evaluate(X_va, y_va)

In [ ]:
# Lag-Llama 
!git clone https://github.com/time-series-foundation-models/lag-llama.git
!pip install -r lag-llama/requirements.txt

# Descargar el checkpoint
from huggingface_hub import hf_hub_download
ckpt_path = hf_hub_download(
    repo_id="time-series-foundation-models/Lag-Llama",
    filename="lag-llama.ckpt"